# Recurrent Neural Net (RNN)
Now we will build a (time-series) recurrent neural net to do sentiment classification with the IMDB Movie reviews dataset.

## Imports
First, let's import everything we'll need.

In [9]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM, RNN, GRU
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Bidirectional
from keras.layers import Embedding, BatchNormalization
from keras.preprocessing import sequence
from keras.utils import np_utils

## Load Data
Now let's load the mnist data.

In [8]:
num_classes = 2
max_idx = 10000

# Constrain dataset to only select top 10000 words
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_idx)

print("x_train shape is: ", x_train.shape)
print("y_train shape is: ", y_train.shape)

print("x_test shape is: ", x_test.shape)
print("y_test shape is: ", y_test.shape)


#save this for later
labels = y_test

x_train shape is:  (25000,)
y_train shape is:  (25000,)
x_test shape is:  (25000,)
y_test shape is:  (25000,)


Let's first take a quick look at what our data looks like:

In [52]:
print("X_train set:", x_train[0])
print("The length of this review is: ", len(x_train[0]))
print("Y_train set:", y_train [0])

X_train set: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
The length of this review is:  218
Y_train set:

As you can see, the features training set contains reviews encoded as a sequences of integers (indexed by frequency in the data set), and the label is a '1' for a positive review, and '0' for a negative review.

Because of this, we will later add an embedding layer that maps each word to an "embedded" vector.

## Format Data
In order to feed the data into a neural network, we must make sure that all training and testing examples are the same length.

In [10]:
# Turn all training examples to length of 400
max_len = 400

x_train = sequence.pad_sequences(x_train, max_len)
x_test = sequence.pad_sequences(x_test, max_len)

print("new x_train shape is: ", x_train.shape)
print("new x_test shape is: ", x_test.shape)

new x_train shape is:  (25000, 400)
new x_test shape is:  (25000, 400)


## Build Model
Now we can build our RNN/LSTM model.

In [59]:
# Create model
model = Sequential()

# Add embedding to words
model.add(Embedding(input_dim=max_idx+1, output_dim=64,
                    input_length=max_len))

# Add 1D Convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Dropout(.15))
model.add(MaxPooling1D(pool_size=2))

# Add bidirectional LSTM layer
model.add(Bidirectional(LSTM(16, return_sequences=False, recurrent_dropout=.15)))
model.add(Dropout(.15))

# Add time-ordered dense layer
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))

# Add final softmax layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Lets take a quick look at our model:

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 400, 64)           640064    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 400, 32)           6176      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 200, 32)           0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 32)                6272      
_________________________________________________________________
dropout_26 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_9 (Batch (None, 64)                256       
__________

## Train Model
Now let's fit the model.

In [61]:
model.fit(x_train, y_train, batch_size=128,
          epochs=5, verbose=1, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 86s 3ms/step - loss: 0.4924 - acc: 0.7405 - val_loss: 0.3500 - val_acc: 0.8442
Epoch 2/5
25000/25000 [==============================] - 82s 3ms/step - loss: 0.2778 - acc: 0.8880 - val_loss: 0.3297 - val_acc: 0.8657
Epoch 3/5
25000/25000 [==============================] - 85s 3ms/step - loss: 0.1779 - acc: 0.9326 - val_loss: 0.3775 - val_acc: 0.8373
Epoch 4/5
25000/25000 [==============================] - 84s 3ms/step - loss: 0.1082 - acc: 0.9619 - val_loss: 0.4378 - val_acc: 0.8385
Epoch 5/5
25000/25000 [==============================] - 84s 3ms/step - loss: 0.0551 - acc: 0.9812 - val_loss: 0.5160 - val_acc: 0.8436


## Evaluate Model
Let's see how well our model did.

In [62]:
score = model.evaluate(x_test, y_test)

print("Test score:", score[0])
print("Test accuracy:", score[1])

25000/25000 [==============================] - 26s 1ms/step
Test score: 0.5160399737548829
Test accuracy: 0.84356


Clearly, we get pretty solid results (though it does look like this model is overfitting).

## Compare to Regular Neural Network

Let's see how this would compare to a regular Dense neural network.

In [16]:
# Create Model
model = Sequential()

# Create first dense layer
model.add(Dense(512, input_shape=(400,)))
model.add(Activation('relu'))

# Create second dense layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Create third dense layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Create final sigmoid layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Now fit the model:

In [17]:
model.fit(x_train, y_train, batch_size=128,
          epochs=5, verbose=1, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 7s 287us/step - loss: 0.7414 - acc: 0.5159 - val_loss: 0.7645 - val_acc: 0.5093
Epoch 2/5
25000/25000 [==============================] - 7s 263us/step - loss: 0.6776 - acc: 0.5754 - val_loss: 0.7074 - val_acc: 0.5127
Epoch 3/5
25000/25000 [==============================] - 7s 263us/step - loss: 0.6340 - acc: 0.6305 - val_loss: 0.7288 - val_acc: 0.5123
Epoch 4/5
25000/25000 [==============================] - 7s 266us/step - loss: 0.5631 - acc: 0.7006 - val_loss: 0.7868 - val_acc: 0.5153
Epoch 5/5
25000/25000 [==============================] - 7s 260us/step - loss: 0.4717 - acc: 0.7652 - val_loss: 0.9145 - val_acc: 0.5108


In [18]:
score = model.evaluate(x_test, y_test)

print("Test score:", score[0])
print("Test accuracy:", score[1])

25000/25000 [==============================] - 3s 120us/step
Test score: 0.9144534546279908
Test accuracy: 0.5108


Wow - what a difference between a vanilla neural net and the recurrent neural net. The regular neural net barely yields an accuracy over 50%.